In [2]:
import pandas as pd
import numpy as np
import dateutil.tz
import datetime as dt
import json

from io import StringIO
import urllib3
import logging
import sys
from json import dumps
import time
import glob
def get_ymd(datetime):
    year = datetime.year
    month = datetime.month
    day = datetime.day
            
    if month < 10:
        month = '0' + str(month)
    if day < 10:
        day = '0' + str(day)
    return year, month, day

def first_day_next_month(date):
    return (date.replace(day=1) + dt.timedelta(days=32)).replace(day=1)

def last_second_of_month(date: str) -> str:
    return str((pd.event_timestamp(date) + pd.offsets.MonthEnd(0)).date()) + " 23:59:59"

def first_second_of_month(date: str) -> str:
    return str((pd.event_timestamp(date) + pd.offsets.MonthBegin(0)).date()) + " 00:00:00"

streamer = StringIO()

def setup_logging():
    logger = logging.getLogger()
    for h in logger.handlers:
        logger.removeHandler(h)
     
    h = logging.StreamHandler(stream = streamer)
    h.setFormatter(logging.Formatter("%(asctime)s %(levelname)s: %(message)s",
                              "%Y-%m-%d %H:%M:%S"))
    logger.addHandler(h)
    logger.setLevel(logging.INFO)
    return logger

def query_log(query_id, table, logger):
    status = wr.athena.get_query_execution(query_id)['Status']['State']
    if wr.athena.get_query_execution(query_id)['Status']['State'] in ['FAILED', 'CANCELLED']:
        logger.critical(table + ': query is in ' + status + ' State. ' + 'QueryID: ' + query_id)
    else:
        logger.info(table + ': query is in ' + status + ' State. ' + 'QueryID: ' + query_id)
    return None


In [6]:
all_files = glob.glob('../cibil_data/parsed_data_v2/parsed_data/*.csv')

In [11]:
df_val = []
for file in all_files:
    df_val.append(pd.read_csv(file))

In [12]:
df_val = pd.concat(df_val)

In [13]:
df_val = df_val[df_val['output'].notna()]

In [14]:
def get_cibil_score(cibil_info):
    t=0
    for item in eval(cibil_info):
        if item.isnumeric():
            t=int(item)
            return int(item)
    if t==0:
        print(cibil_info)
    return -1

In [15]:
df_val['cibil_score'] = df_val['cibil_info_with_factors'].apply(lambda x:get_cibil_score(x))

['CIBILTUSC3', '-1']


In [16]:
df_val = df_val[df_val['cibil_score'].notna()]

In [27]:
def map_bb_files_with_val(df_bb,df_val):
    
    dfs=[]
    for i,r in df_val.iterrows():
        name=phone=cibil_score=output=is_ok=''
        name=r['name']
        uid = eval(r['user_identifier'])
        pan_id = ''
        if 'PAN' in uid:
            pan_id=uid['PAN']
        phone_nos = eval(r['phone_no'])
        
        if pan_id !='':
            is_ok_l = df_bb[df_bb['Borrower PAN']==pan_id]['TS 3: BB Sanctioned'].tolist()
            if len(is_ok_l)>0:
                print(is_ok_l)
                is_ok=is_ok_l[0]
                
        elif len(phone_nos)>0:
#             print(phone_nos)
            for ph_no in phone_nos:
                t=df_bb[df_bb['Applicant Mob #']==ph_no]['TS 3: BB Sanctioned'].tolist()
                if len(t)>0:
                    is_ok=t[0]
                    break
        else:
            pass
                

        temp_df = pd.DataFrame({'name':[name],'cibil_score':[r['cibil_score']],'output':[r['output']], 'is_ok_bb':[is_ok]}
                              )
        dfs.append(temp_df)
    return pd.concat(dfs)
        
        
        
    

In [19]:
# v1 = map_bb_files_with_val(df_bb,df_val)

In [3]:
df_bb = pd.read_csv('../cibil_data/bb_val_sample/FS OwnBook - Loan Status Tracker Nov\'22 (Confidential) - 3.Cibil Check Master.csv')

In [4]:
df_bb.columns = df_bb.iloc[0]

In [5]:
df_bb['Borrower Name'] = df_bb['Borrower Name'].apply(lambda x:x.strip() if type(x)=='str' else x)

In [20]:
df_bb1 = pd.read_csv('../cibil_data/bb_val_sample/FS OwnBook - Loan Status Tracker Nov\'22 (Confidential) - 5.Files Master.csv')

In [21]:
df_bb1 = df_bb1[['Back to Index','Unnamed: 1','Unnamed: 2','Unnamed: 27']].dropna()

In [22]:
df_bb1.columns = df_bb1.iloc[0]

In [23]:
df_bb1 = df_bb1[1:]

In [24]:
df_merge = df_bb.merge(df_bb1,left_on='Applicant Mob #',right_on='Applicant Mobile Number')

In [29]:
df_map = pd.read_csv('../cibil_data/mapped_truck_data.csv')

In [31]:
test_list = glob.glob('/home/ubuntu/fin_services/cibil_data/parsed_data/*.csv')

In [34]:
gps_trucks = pd.read_csv('../trucks_idle_profiling/gps_idle_ratio.csv')['truck_number'].tolist()

In [73]:
ldf=[]
for item in test_list:
    
    df_temp = pd.read_csv(item)
    pdf_link = item.replace('parsed_data','test/cibil_pdf').replace('.csv','.PDF')
    phone_nos = eval(df_temp['phone_no'][0])
    uid = eval(df_temp['user_identifier'][0])
    pan_id = ''
    if 'PAN' in uid:
        pan_id=uid['PAN']
    truck_no = ''
    is_verified=False
    if (len(phone_nos))>0:
        for mob_no in phone_nos:
            tk_list=df_merge[df_merge['Applicant Mob #']==mob_no]['Truck Number'].tolist()
            if len(tk_list)>0:
                truck_no=tk_list[0]
                t = df_merge[df_merge['Applicant Mob #']==mob_no]['TS 3: BB Sanctioned'].tolist()
                if len(t)>0:
                    is_verified=True
    fo_id=''
    
    if truck_no !='':
        fo_id = df_map[df_map['truck_number']==truck_no]['fleet_owner_id'].tolist()
        if len(fo_id)>0:
            fo_id=fo_id[0]
        else:
            fo_id=''
    else:
        for mob_no in phone_nos:
            mapped_tk_list = df_map[df_map['mobile_no']==mob_no]['truck_number'].tolist()
            if len(mapped_tk_list)>0:
                truck_no=mapped_tk_list[0]
    is_gps = False
    if truck_no in  gps_trucks:
        is_gps=True
            
    
    
    ldf.append(pd.DataFrame({'truck_number':[truck_no],'mobile_no':[phone_nos],'pdf_link':[pdf_link],'pan_id':[pan_id],'fo_id':[fo_id],'is_approved':[is_verified],'is_train':[False],'is_gps':[is_gps]}))
        
    

In [74]:
test_df = pd.concat(ldf)

In [75]:
train_link =  glob.glob('/home/ubuntu/fin_services/cibil_data/train/cibil_out/*.csv') +  glob.glob('/home/ubuntu/fin_services/cibil_data/train/new_cibil_out/*.csv')


In [90]:
pd.read_csv('../cibil_data/train/cibil_out/SHAIK SHAKEEL CIBIL_Report - Shaik Wassi.csv')

,name,gender,cibil_info_with_factors,cibil_name,user_identifier,dob,address,phone_no,email,acc_summary,account_info,enquiry,account_info_new
0,SK SHAKEEL,MALE,"['CIBILTUSC3', '-1']",CIBILTUSC3,"{'PAN': 'HWDPS6622A', 'UNIVERSAL ID': '9283493...",01/12/1998,[{'address': ' 10 56 2 GOLLA BASTI MEDCHAL MED...,['7989314960'],[],"{'total': ' 0', 'overdue': ' 0', 'zero_balance...",[],"[{'member': 'NOT DISCLOSED', 'enquiry_date': '...",[]


In [88]:
ldf=[]
for item in train_link:
    
    df_temp = pd.read_csv(item)
    pdf_link=''
    if 'cibil_out' in item.split('/'):
        pdf_link = item.replace('train/cibil_out','train/cibil_pdf').replace('.csv','.PDF')
    elif 'new_cibil_out' in item.split('/'):
        pdf_link = item.replace('new_cibil_out','new_cibil_pdf').replace('.csv','.PDF')
    else:
        print(item)

        
    phone_nos = eval(df_temp['phone_no'][0])
    uid = eval(df_temp['user_identifier'][0])
    pan_id = ''
    if 'PAN' in uid:
        pan_id=uid['PAN']
    truck_no = ''
    is_verified=False
    if (len(phone_nos))>0:
        for mob_no in phone_nos:
            tk_list=df_merge[df_merge['Applicant Mob #']==mob_no]['Truck Number'].tolist()
            if len(tk_list)>0:
                truck_no=tk_list[0]
                t = df_merge[df_merge['Applicant Mob #']==mob_no]['TS 3: BB Sanctioned'].tolist()
                if len(t)>0:
                    is_verified=True
                
    fo_id=''
    
    if truck_no !='':
        fo_id = df_map[df_map['truck_number']==truck_no]['fleet_owner_id'].tolist()
        if len(fo_id)>0:
            fo_id=fo_id[0]
        else:
            fo_id=''
    else:
        for mob_no in phone_nos:
            mapped_tk_list = df_map[df_map['mobile_no']==mob_no]['truck_number'].tolist()
            if len(mapped_tk_list)>0:
                truck_no=mapped_tk_list[0]
    is_gps = False
    if truck_no in  gps_trucks or truck_no in ['AP09TB3411','AP29TB8644']:
        is_gps=True
            
    
    
    ldf.append(pd.DataFrame({'truck_number':[truck_no],'mobile_no':[phone_nos],'pdf_link':[pdf_link],'pan_id':[pan_id],'fo_id':[fo_id],'is_approved':[is_verified],'is_train':[True],'is_gps':[is_gps]}))
        
    

In [89]:
train_df = pd.concat(ldf)

In [78]:
df_all = pd.concat([test_df,train_df])

In [79]:
df_all.reset_index(inplace=True)

In [80]:
df_all.drop(['index'],axis=1,inplace=True)

In [81]:
df_all.to_csv('../cibil_data/cibil_bb_info_pdf_link.csv',index=False)

In [82]:
!ls '/home/ubuntu/fin_services/cibil_data/test/cibil_pdf/NAVNATH HANUMANT PHADKE CIBIL_Report - Shaik Wassi.PDF'

'/home/ubuntu/fin_services/cibil_data/test/cibil_pdf/NAVNATH HANUMANT PHADKE CIBIL_Report - Shaik Wassi.PDF'


In [83]:
# !head '/home/ubuntu/fin_services/cibil_data/cibil_bb_info_pdf_link.csv'

In [84]:
df_all[df_all['is_gps']]

,truck_number,mobile_no,pdf_link,pan_id,fo_id,is_approved,is_train,is_gps
717,AP09TB3411,[9010107400],/home/ubuntu/fin_services/cibil_data/train/new...,,,True,True,True
1056,AP09TB3411,"[9010107400, 9010107400, 7702104150, 7702104150]",/home/ubuntu/fin_services/cibil_data/train/new...,AQLPD4585A,,True,True,True


In [85]:
df_all[df_all['is_gps']]['pdf_link'].tolist()

['/home/ubuntu/fin_services/cibil_data/train/new_cibil_pdf/DONTHI DURGABAI - Kala Venkatarambabu.PDF',
 '/home/ubuntu/fin_services/cibil_data/train/new_cibil_pdf/D MADHU SUDHAN REDDY - Kala Venkatarambabu.PDF']

In [86]:
# train_df[train_df['truck_number']=='AP29TB8644']

In [92]:
 df_all[df_all['truck_number']=='AP29TB8644']

,truck_number,mobile_no,pdf_link,pan_id,fo_id,is_approved,is_train,is_gps
